In [1]:
%%time
import warnings
warnings.filterwarnings('ignore')
from collections import  namedtuple
import pandas as pd
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from itertools import product

from video699.screen.semantic_segmentation.fastai_detector import *
from video699.screen.semantic_segmentation.common import *
from video699.screen.semantic_segmentation.evaluation import *

CPU times: user 5.82 s, sys: 1.23 s, total: 7.05 s
Wall time: 23.4 s


In [2]:
resize_factor = [8]
unfrozen_epochs = [4, 7]
frozen_epochs = [4, 7]
base_lower_bound = [5, 7, 10, 15]
erosion_dilation_kernel_size = [20, 50, 80, 150]
ratio_split_lower_bound = [0.3, 0.4, 0.5, 0.7, 0.8, 0.9]

In [3]:
train = list(product(resize_factor, frozen_epochs, unfrozen_epochs))
train_names = ['resize_factor', 'frozen_epochs', 'unfrozen_epochs']

post_processing = list(product(base_lower_bound, erosion_dilation_kernel_size, ratio_split_lower_bound))
post_processing_names = ['base_lower_bound', 'erosion_dilation_kernel_size', 'ratio_split_lower_bound']

all_lectures = [video.filename for video in ALL_VIDEOS]
all_frames = [frame for video in ALL_VIDEOS for frame in video]
all_frames_grouped_by_videos = {video.filename: [frame for frame in video] for video in ALL_VIDEOS}

detector = FastAIScreenDetector()
actual_detector = AnnotatedSampledVideoScreenDetector()

In [4]:
def filtered_by(name, used):
    return any([lecture in str(name) for lecture in used]) and 'frame' in str(name)

def split_by(name, validation):
    return any([lecture in str(name) for lecture in validation])

In [5]:
def model_selection(lectures, train_names, post_processing_names, default_filtered_by, default_split_by):
    def make_splits(lectures):
        Split = namedtuple('Split', ['train', 'valid'])
        kf = KFold(n_splits=5, shuffle=True, random_state=123)
        splits = {}
        for j, split in enumerate(kf.split(lectures)):    
            train_lectures = [lectures[index] for index in split[0]]
            valid_lectures = [lectures[index] for index in split[1]]
            splits[j] = Split(train=train_lectures, valid=valid_lectures)
        return splits

    splits = make_splits(all_lectures)
    df_all = pd.DataFrame(columns=train_names + post_processing_names + ['iou', 'wrong_count', 'kfold_split'])

    for train_values in tqdm(train):
        resize_factor, frozen_epochs, unfrozen_epochs = train_values
        CONFIGURATION['resize_factor'] = str(resize_factor)
        CONFIGURATION['frozen_epochs'] = str(frozen_epochs)
        CONFIGURATION['unfrozen_epochs'] = str(unfrozen_epochs)

        for j in splits.keys():
            filtered_by = partial(default_filtered_by, used=splits[j].train + splits[j].valid)
            split_by = partial(default_split_by, validation=splits[j].valid)

            detector = FastAIScreenDetector(filtered_by=filtered_by, valid_func=split_by)
            detector.train()

            valid_frames = [frame for frame in all_frames if split_by(frame.pathname)]
            actuals = [actual_detector.detect(frame) for frame in valid_frames]
            sem_preds = detector.semantic_segmentation_batch(valid_frames)

            for post_processing_values in post_processing:    
                preds = detector.post_processing_batch(sem_preds, valid_frames, **dict(zip(post_processing_names, post_processing_values)))
                wrong_count, ious, _ = evaluate(actuals, preds)

                iou_score = np.nanmean(ious)
                wrong_count = len(wrong_count)
                df_all.loc[len(df_all)] = train_values + post_processing_values + (iou_score, wrong_count, j)
    return df_all

In [6]:
def convert_params(best_params):
    converted_params = []
    for i, par in enumerate(best_params):
        if par.is_integer():
            converted_params.append(int(par))
        elif isinstance(par, np.int64) or isinstance(par, np.float64):
            converted_params.append(par.item())
        else:
            converted_params.append(par)
    best_params = tuple(converted_params)
    
    return best_params

### Lecture-wise 5-fold cross validation

In [8]:
kf = KFold(n_splits=5, shuffle=True, random_state=123)
df_best_models = pd.DataFrame(columns=train_names + post_processing_names + ['iou', 'wrong_count'])

for i, split in tqdm(enumerate(list(kf.split(all_lectures))[2:])):
    print(f"###################### Split No. {i}")
    other_lectures = [all_lectures[index] for index in split[0]]
    test_lectures = [all_lectures[index] for index in split[1]]
    
    # Model selection
    df_all = model_selection(other_lectures, train_names, post_processing_names, filtered_by, split_by)    
    df_all['wrong_count'] = df_all['wrong_count'].astype(int)
    best_params = df_all.groupby(train_names + post_processing_names).mean().sort_values(by=['wrong_count', 'iou']).iloc[0].name
    best_params = convert_params(best_params)
    best_params = dict(zip(train_names + post_processing_names, best_params))
    test_filtered_by = partial(filtered_by, used=all_lectures)
    test_split_by = partial(split_by, validation=test_lectures)
    
    
    best_detector = FastAIScreenDetector(filtered_by=test_filtered_by, valid_func=test_split_by)
    best_detector.train(**best_params)

    test_frames = [frame for frame in all_frames if test_split_by(frame.pathname)]
    actuals = [actual_detector.detect(frame) for frame in test_frames]
    preds = [best_detector.detect(frame) for frame in test_frames]
    
    wrong_count, ious, _ = evaluate(actuals, preds)
    iou_score = np.nanmean(ious)
    wrong_count = len(wrong_count)
    df_best_models.loc[len(df_best_models)] = tuple(best_params.values()) + (iou_score, wrong_count)
    df_best_models.to_csv('cross_validation_results_lecture_wise-factor_8_2.csv', index=False)
    
df_best_models.to_csv('cross_validation_results_lecture_wise-factor-8_2.csv', index=False)

0it [00:00, ?it/s]
  0%|          | 0/4 [00:00<?, ?it/s]

###################### Split No. 0


epoch,train_loss,valid_loss,acc,dice,iou_sem_seg,time
0,0.286771,0.067948,0.974592,0.952785,0.910352,00:06
1,0.153249,0.048038,0.979800,0.962952,0.928865,00:06
2,0.089731,0.039526,0.983017,0.968302,0.938958,00:06
3,0.059652,0.037621,0.983826,0.969042,0.940509,00:06


epoch,train_loss,valid_loss,acc,dice,iou_sem_seg,time
0,0.039549,0.044262,0.983299,0.968229,0.939812,00:06
1,0.034962,0.044233,0.984169,0.969062,0.941241,00:06
2,0.025432,0.040899,0.985160,0.970638,0.944170,00:06
3,0.021466,0.035308,0.987126,0.975103,0.951964,00:06


epoch,train_loss,valid_loss,acc,dice,iou_sem_seg,time
0,0.214857,0.094618,0.964432,0.887962,0.819887,00:06
1,0.150119,0.162232,0.951734,0.771377,0.732372,00:06
2,0.087151,0.152566,0.961977,0.824062,0.782913,00:06
3,0.059120,0.171951,0.957762,0.787524,0.757766,00:06


...

epoch,train_loss,valid_loss,acc,dice,iou_sem_seg,time
0,0.017856,0.054583,0.980527,0.971476,0.945401,00:07
1,0.021380,0.072793,0.976722,0.966537,0.938133,00:07
2,0.022750,0.056683,0.981682,0.972757,0.947836,00:07
3,0.027237,0.052984,0.981506,0.972850,0.948579,00:07
4,0.022236,0.060737,0.980783,0.971717,0.946050,00:07
5,0.015944,0.048852,0.984449,0.977228,0.956075,00:07
6,0.013856,0.041516,0.986922,0.980791,0.962688,00:07



100%|██████████| 4/4 [38:43<00:00, 580.83s/it]


epoch,train_loss,valid_loss,acc,dice,iou_sem_seg,time
0,0.245378,0.112961,0.956554,0.931016,0.877270,00:07
1,0.167122,0.697366,0.836202,0.613150,0.501707,00:06
2,0.115428,0.071861,0.973106,0.959694,0.923764,00:06
3,0.072980,0.195744,0.944312,0.907162,0.843738,00:07
4,0.048571,0.056799,0.979570,0.969490,0.942240,00:06
5,0.035903,0.066726,0.979433,0.969403,0.941986,00:06
6,0.025859,0.069277,0.978097,0.967251,0.938033,00:06


epoch,train_loss,valid_loss,acc,dice,iou_sem_seg,time
0,0.020169,0.049757,0.982149,0.973833,0.949773,00:07
1,0.026202,0.077717,0.975132,0.963458,0.932335,00:07
2,0.022665,0.093077,0.976061,0.964643,0.934426,00:07
3,0.019178,0.098031,0.976268,0.963963,0.932972,00:07
4,0.015599,0.060968,0.983724,0.975984,0.954268,00:07
5,0.013120,0.087722,0.976120,0.962946,0.930821,00:06
6,0.011414,0.120143,0.970249,0.952445,0.912396,00:06


3it [1:57:47, 2355.93s/it]
